# Deep Dive into Fragrances, Part 2: Fragrance Recommendations 
### Why this Project? 
This project is in two parts: An analysis of fragrance notes in various fragrances and user-preferred fragrance notes, and a recommendation system that gives 5 fragrance recommendations based on preferred fragrance notes. 

This project holds significant importance in the realm of data science and consumer preferences. By meticulously analyzing fragrance notes and comparing them to individual preferences, it provides valuable insights into the intricate world of scent preferences. This not only enhances our understanding of consumer behavior but also empowers businesses in the fragrance industry to tailor their offerings to specific demographics. Furthermore, the recommendation system introduces a personalized element, revolutionizing how consumers discover and select fragrances. This innovation has the potential to revolutionize the fragrance market, creating a more tailored and satisfying experience for consumers while offering businesses the opportunity to refine their product offerings. Overall, this project not only showcases the power of data-driven insights but also addresses a tangible need in a thriving industry.

### Goal (For this Notebook)
Create a Recommendation system that will recommend fragrances to users based on their fragrance note preferences.

### Data Source (see previous notebook for more):
Kaggle: <br>
https://www.kaggle.com/datasets/nandini1999/perfume-recommendation-dataset

Web Scraping/Information hard-coded from Fragrantica

In [53]:
#import dependencies
import pandas as pd
import numpy as np 
import sklearn 
import scipy
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_colwidth', None)

In [54]:
#import data
df = pd.read_csv('full_perfume_data.csv')
df.columns

Index(['Unnamed: 0', 'Name', 'Brand', 'Description', 'Notes',
       'extracted_notes', 'categories'],
      dtype='object')

In [55]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [56]:
df.head()

,Name,Brand,Description,Notes,extracted_notes,categories
0,Tihota Eau de Parfum,Indult,"Rapa Nui for sugar, Tihota is, quite simply, The One. The One that will call to you every moment you're not smelling it, The One that you've only had hints of in other vanilla perfumes, The One that lasts and lasts, The One that has perfectly captured the essence of the pure nature of fresh vanilla beans and has harnessed it beyond your wildest dreams. It inspires obsessive devotion with its fragrance of smooth, vanilla bean pods dipped in honeyed water and left to steep. The result is the pure magic, an unbridled vanilla, sweet, raw and achingly desirable. Of course, Tihota isn't the first perfume to focus on the dark sweetness of vanilla... perfumery is filled with vanillas, but this smells like the dream of a master perfumer who was obsessed with vanilla and was finally given free rein to worship the note with no apologies. It is, without a sliver of a doubt, the finest pure vanilla we've ever had the honor of carrying, and it's so beautiful it pains us to put the bottle down.","Vanilla bean, musks","Vanilla, Honey, Sugar, Fir, Water","['spice', 'sweet/gourmand']"
1,Sola Parfum,Di Ser,"A tribute to the expanse of space extending from the sky to the flower and fruit laden earth below, Sola, the newest offering from DI SER and perfumer Yasuyuki Shinohara, showcases the natural flora of Hokkaido, Japan with locally sourced and sustainably harvested ingredients from DI SERs own gardens. Starting with aromatic notes of freshly harvested Hokkaido lavender, lemongrass and yuzu, the uplifting scent moves to a floral middle of Japanese rose, magnolia, geranium and jasmine before settling on a spiritual bed of frankincense and myrrh. The spiritual and meditative scent evokes an imagery of a peaceful universe in complete harmony with the bounties of nature and of peace and contentment with its generous offerings.","Lavender, Yuzu, Lemongrass, Magnolia, Geranium, Jasmine, Frankincense, Myrrh","Lemon, Lemongrass, Yuzu, Pea, Geranium, Lavender, Magnolia, Rose, Jasmine, Aromatic Notes, Grass, Incense, Myrrh",['citrus']
2,Kagiroi Parfum,Di Ser,"An aromatic ode to the ancient beauty of Japan, Kagiroi represents the changes in the shade of the dawn sky over the mountains of Hokkaido, Japan. As the dawn sky transitions through shades of purple, yellow, orange and red, this scent that showcases many ingredients used for the first time in perfumery transitions from an aromatic slightly spicy top to an earthy and woody base, representing the ebb and flow in the stages of life. The perfume opens with notes of Japanese herbs and citruses, moves to a slightly floral spicy middle and settles into a deep woody base accentuated by the use of the Hinoki cedar that dot the mountain landscapes. A deeply personal and spiritual scent, Kagiroi is a scent of shifting aromatic complexities representing transitions in the journey of life.Parfum Edition: We are honored and delighted to offer the rare and precious Parfum edition of Kagiroi, which adds to the composition a precious specimen of Japanese agarwood, which adds rich, spiritual dimensionality and a truly impressive and long-lasting drydown to this already complex and wonderful scent.","Green yuzu, green shikuwasa, sansho seed, coriander, ylang-ylang, shiso, rosewood, vetiver, hinoki, cypriol, patchouli, agarwood","Citruses, Orange, Cedar, Fir",['citrus']
3,Velvet Fantasy Eau de Parfum,Montale,"Velvet Fantasy is a solar fragrance where citrus and velvety fruits mingle with the elegance of rose and the gourmand femininity of violet. Thereafter, incense unveils amber and white musk to create a bewitching and wild dance. What We Think A sweet and fruity fragrance with a rich long lasting warmth. Like a decadent dessert, it starts off bringing together the bright freshness of the tangerine with some sweet berry and apple notes and the creaminess of the toffee and vanilla. The fragrance smoothly transitions into a delicio

In [57]:
#fix the notes column setup so that there's no leading whitespace
df = df.applymap(lambda x: x.lstrip() if isinstance(x, str) else x)
df['Notes'].loc[0] #Check to see if it works

'Vanilla bean, musks'

In [58]:
df.isna().sum()

Name                0
Brand               0
Description         0
Notes               9
extracted_notes    18
categories          2
dtype: int64

In [59]:
df.dropna(inplace = True)

In [60]:
df.isna().sum()

Name               0
Brand              0
Description        0
Notes              0
extracted_notes    0
categories         0
dtype: int64

In [61]:
df['Notes_list'] = df['Notes'].apply(lambda x: x.split(','))

In [62]:
df['extracted_list'] = df['extracted_notes'].apply(lambda x: x.split(','))

In [63]:
df.head()

,Name,Brand,Description,Notes,extracted_notes,categories,Notes_list,extracted_list
0,Tihota Eau de Parfum,Indult,"Rapa Nui for sugar, Tihota is, quite simply, The One. The One that will call to you every moment you're not smelling it, The One that you've only had hints of in other vanilla perfumes, The One that lasts and lasts, The One that has perfectly captured the essence of the pure nature of fresh vanilla beans and has harnessed it beyond your wildest dreams. It inspires obsessive devotion with its fragrance of smooth, vanilla bean pods dipped in honeyed water and left to steep. The result is the pure magic, an unbridled vanilla, sweet, raw and achingly desirable. Of course, Tihota isn't the first perfume to focus on the dark sweetness of vanilla... perfumery is filled with vanillas, but this smells like the dream of a master perfumer who was obsessed with vanilla and was finally given free rein to worship the note with no apologies. It is, without a sliver of a doubt, the finest pure vanilla we've ever had the honor of carrying, and it's so beautiful it pains us to put the bottle down.","Vanilla bean, musks","Vanilla, Honey, Sugar, Fir, Water","['spice', 'sweet/gourmand']","[Vanilla bean, musks]","[Vanilla, Honey, Sugar, Fir, Water]"
1,Sola Parfum,Di Ser,"A tribute to the expanse of space extending from the sky to the flower and fruit laden earth below, Sola, the newest offering from DI SER and perfumer Yasuyuki Shinohara, showcases the natural flora of Hokkaido, Japan with locally sourced and sustainably harvested ingredients from DI SERs own gardens. Starting with aromatic notes of freshly harvested Hokkaido lavender, lemongrass and yuzu, the uplifting scent moves to a floral middle of Japanese rose, magnolia, geranium and jasmine before settling on a spiritual bed of frankincense and myrrh. The spiritual and meditative scent evokes an imagery of a peaceful universe in complete harmony with the bounties of nature and of peace and contentment with its generous offerings.","Lavender, Yuzu, Lemongrass, Magnolia, Geranium, Jasmine, Frankincense, Myrrh","Lemon, Lemongrass, Yuzu, Pea, Geranium, Lavender, Magnolia, Rose, Jasmine, Aromatic Notes, Grass, Incense, Myrrh",['citrus'],"[Lavender, Yuzu, Lemongrass, Magnolia, Geranium, Jasmine, Frankincense, Myrrh]","[Lemon, Lemongrass, Yuzu, Pea, Geranium, Lavender, Magnolia, Rose, Jasmine, Aromatic Notes, Grass, Incense, Myrrh]"
2,Kagiroi Parfum,Di Ser,"An aromatic ode to the ancient beauty of Japan, Kagiroi represents the changes in the shade of the dawn sky over the mountains of Hokkaido, Japan. As the dawn sky transitions through shades of purple, yellow, orange and red, this scent that showcases many ingredients used for the first time in perfumery transitions from an aromatic slightly spicy top to an earthy and woody base, representing the ebb and flow in the stages of life. The perfume opens with notes of Japanese herbs and citruses, moves to a slightly floral spicy middle and settles into a deep woody base accentuated by the use of the Hinoki cedar that dot the mountain landscapes. A deeply personal and spiritual scent, Kagiroi is a scent of shifting aromatic complexities representing transitions in the journey of life.Parfum Edition: We are honored and delighted to offer the rare and precious Parfum edition of Kagiroi, which adds to the composition a precious specimen of Japanese agarwood, which adds rich, spiritual dimensionality and a truly impressive and long-lasting drydown to this already complex and wonderful scent.","Green yuzu, green shikuwasa, sansho seed, coriander, ylang-ylang, shiso, rosewood, vetiver, hinoki, cypriol, patchouli, agarwood","Citruses, Orange, Cedar, Fir",['citrus'],"[Green yuzu, green shikuwasa, sansho seed, coriander, ylang-ylang, shiso, rosewood, vetiver, hinoki, cypriol, patchouli, agarwood]","[Citruses, Orange, Cedar, Fir]"
3,Velvet Fantasy Eau de Parfum,Montale,"Velvet Fantasy is a solar fragrance where citrus and velvety fruits mingle with 

In [64]:
#df['Brand'].unique()

In [65]:
#df[df['Brand'] == 'Phlur']

Now, go through the extracted notes and notes lists, and get a list of all of the notes together (actually, make it a set for distinct values). Use these notes to get the notes of interest. <br> 

MAYBE only use extracted lists, because those are from the descriptions and are really what the brands want the consumer to focus on (those should be the notes that "stick out" the most

In [66]:
#get all the notes found in "extracted_list" column, and all the notes found in "notes_list" columns
def get_all_notes(df): 
    og_notes = df['Notes_list']
    extracted_notes = df['extracted_list']
    
    all_notes_og = []
    all_notes_ex = []
    
    for og_lst in og_notes: #for each list in the og notes column
        for og_note in og_lst: #for each note in the list
            og_note = og_note.lstrip()
            if og_note not in all_notes_og: 
                all_notes_og.append(og_note)
                
    for ex_lst in extracted_notes: 
        for ex_note in ex_lst: 
            ex_note = ex_note.lstrip()
            if ex_note not in all_notes_ex: 
                all_notes_ex.append(ex_note)
                
    return all_notes_og, all_notes_ex


In [67]:
#get_all_notes(df)[1]

In [68]:
len(get_all_notes(df)[1])

631

In [69]:
len(get_all_notes(df)[0])

4001

General outline of how I want this recommendation system set up: <br> 
1. Ask user which note category are they interested in
2. Ask the user which specific notes in the category they are interested in, add the note to a list
3. Then ask the user if they want to add more notes; if 'yes', then let them choose a category and then let them choose the note from that category. Add this note to the list. 
4. Repeat #3 for a set amount of times
5. Use cosine similarity to combine the accumulated list of user's preferred notes to each fragrances's list of notes
6. Output the top 5 fragrances that have the highest cosine similarity values. 
    * Output the Name of the fragrance (including the brand), the notes in the fragrance, and the cosine similarity score 
7. If a user has tried one of the fragrances, ask if they would like to provide feedback. If yes, store the feedback
 


Before doing the above, first I want to create a new dictionary containing the same keys as the categories dictionary I imported, and have only the values that are present in the fragrances dataframe. By doing this, it is easier to match the chosen notes with a fragrance present in the dataframe. 

In [70]:
import json 

with open('fragrance_categories.json') as frags: 
    frag_categories = json.load(frags)

In [71]:
all_notes_in_frag_df = get_all_notes(df)[0]

new_dict = {}
for k, v in frag_categories.items():
    new_list = []
    for note in all_notes_in_frag_df: 
        if note in v: 
            new_list.append(note)
    new_dict[k] = new_list

In [72]:
def give_recommendation(category_dictionary, fragrances_df): 
    
    #for use later, get cosine similarity between two lists of strings
    def cosine_sim(a,b): #a and b are lists
        a_join = ' '.join(a)
        b_join = ' '.join(b)

        data = [a_join, b_join]
        count_vectorizer = CountVectorizer()
        vector_matrix = count_vectorizer.fit_transform(data)

        cosine_similarity_matrix = cosine_similarity(vector_matrix)

        return cosine_similarity_matrix[0][1] #similarity score
             
        
    categories = list(category_dictionary.keys()) #all possible categories
    total_categories = len(categories)
    
    ##give the user their options:
    print('Categories:')
    for i in range(total_categories): 
        print(str(i + 1) + '. ' + categories[i])
    ##get the user's input, make sure it is a valid input    
    category_choice = int(input(f'Please choose a category from the above (1-{str(total_categories)}): '))
    while category_choice not in range(1, total_categories + 1): 
        category_choice = int(input('Entry not valid, please enter one of the above categories: '))
    
    category_choice = categories[category_choice - 1]
    ##Get the potential values for the category (ie, the values for the category) 
    category_values = category_dictionary[category_choice]
    
    ##give the user the values to chose from: 
    print('\n' + category_choice + ' fragrance notes:')
    for v in range(len(category_values)): 
        print(str(v + 1) + '. ' + category_values[v])
        
    note_choice = int(input(f'Please choose a fragrance note from above (1-{str(len(category_values))}): '))
    while note_choice not in range(1, len(category_values) + 1): 
        note_choice = int(input('Entry not valid, please enter one of the above'))
        
    note_choice_1 = category_values[note_choice -1] #the name of the choice chosen by the user
    
    ##Keep the note choice (and any subsequent choices) in a list: 
    chosen_notes_list = [note_choice_1]
    
    ##Ask user if they would like to add another note, validate the input:
    another_note = input('Would you like to add another note? y/n: ')
    while another_note not in ['y', 'n']:
        another_note = input('Input invalid. Would you like to add another note? y/n (lowercase): ')
    
    ##if the user wants to add more, let them go through the above again. Keep a limit of 3 more inputs
    if another_note == 'y': 
        limit = 0 
        while limit < 3: 
            print('Categories:')
            for i in range(total_categories): 
                print(str(i + 1) + '. ' + categories[i])
            ##get the user's input, make sure it is a valid input    
            category_choice = int(input(f'Please choose a category from the above (1-{str(total_categories)}): '))
            while category_choice not in range(1, total_categories + 1): 
                category_choice = int(input('Entry not valid, please enter one of the above categories: '))

            category_choice = categories[category_choice - 1]
            ##Get the potential values for the category (ie, the values for the category) 
            category_values = category_dictionary[category_choice]

            ##give the user the values to chose from: 
            print('\n' + category_choice + ' fragrance notes:')
            for v in range(len(category_values)): 
                print(str(v + 1) + '. ' + category_values[v])

            note_choice = int(input(f'\nPlease choose a fragrance note from above (1-{str(len(category_values))}): '))
            while note_choice not in range(1, len(category_values) + 1): 
                note_choice = int(input('\nEntry not valid, please enter one of the above'))

            note_choice_x = category_values[note_choice -1] #the name of the choice chosen by the user

            ##Keep the note choice (and any subsequent choices) in a list: 
            chosen_notes_list.append(note_choice_x)
            
            limit += 1
            
            if limit < 3: #if the limit is still less than 3, ask again:
                ##Ask user if they would like to add another note, validate the input:
                another_note = input('\nWould you like to add another note? y/n: ')
                while another_note not in ['y', 'n']:
                    another_note = input('Input invalid. Would you like to add another note? y/n: ')

                if another_note == 'n': 
                    limit = 3
                
    ##now that I have a list of notes user wants, compare the list of notes to those in the dataframe
    comparison_scores = []
    
    ##iterate through extracted_notes_list, compare it to the current list, put the score into comparison_scores
    ###Use cosine_sim function defined above as the measure of similarity
    
    extracted_notes = fragrances_df['extracted_list'].to_list()
    new_extracted = []
    for notes in extracted_notes:
        new_notes = [n.lower().strip() for n in notes]
        new_extracted.append(new_notes)
    
    for notes_list in new_extracted:
        cosine_sim_score = cosine_sim(notes_list, chosen_notes_list)
        comparison_scores.append(round(cosine_sim_score, 3))
        
    ##make comparison_scores a new column: 
    fragrances_df['cosine_sim_score'] = comparison_scores
    
    ##Now, sort the df by cosine_sim_score descending, output the top 3 names: 
    sorted_df = fragrances_df.sort_values(by = ['cosine_sim_score'], ascending = False)
    sorted_df = sorted_df[sorted_df['cosine_sim_score'] > 0.23]
    names = sorted_df['Name'].to_list()
    brands = sorted_df['Brand'].to_list()
    scores = sorted_df['cosine_sim_score'].to_list()
    notes = sorted_df['extracted_notes'].to_list()
    sorted_df['full_name'] = sorted_df['Name'] + ' by ' + sorted_df['Brand']
    total_name = sorted_df['full_name'].to_list()
    
    brand_and_name = []
    for i in range(len(sorted_df)):
        full_name = f'{names[i]} by {brands[i]},  \nNOTES: {notes[i]}, \nSCORE: {scores[i]} \n'
        brand_and_name.append(full_name)
        
    ##Give top n outputs (will set n = 5 for now)
    n = 5
    recommendations = brand_and_name[:n]
    print('\nYour Recommendations (SCORE closer to 1 = closer to preferences): ')
    #print(brand_and_name[:n])
    i = 1
    for rec in recommendations: 
        print(f'{str(i)}. ' + rec)
        i += 1
    
    give_feedback = input('Would you like to give feedback on any of these perfumes? y/n: ')
    while give_feedback not in ['y', 'Y', 'n', 'N']:
        give_feedback = input('INPUT INVALID. Would you like to give feedback on any of these perfumes? y/n: ')
        
    if give_feedback in ['n', 'N']:
        return 'We hope you enjoy your recommendations!'
    
    else: #if user wants to give feedback:
        which_fragrance_num = input('Which fragrance would you like to give feedback for? Please enter 1 - 5 to choose: ')
        while int(which_fragrance_num) not in range(1, n + 1):
            which_fragrance_num = input(f'ENTRY INVALID. Please choose a fragrance, 1 - {str(n)}: ')
            
        which_fragrance_num = int(which_fragrance_num) - 1 #this will be the index to grab the particular fragrance
        ### Things to do: 
        ### Open the json feedback file, turn into df
        with open('fragrance_feedback.json') as frags_feedback: 
            frag_fb = json.load(frags_feedback)
        frag_fb_df = pd.DataFrame({k:pd.Series(v) for k,v in frag_fb.items()})
        
        ### get the input (feedback) for the fragrance
        ### Using the which_fragrance_num, get the name of the fragrance (in 'Name' by 'Brand')
        chosen_frag = total_name[which_fragrance_num] #gives the full name; will be used to input feedback 
        ### Name of fragrance will be used as the key of the json file dict ^^
        cols = list(frag_fb_df.columns)
        chosen_col = chosen_frag
        temp_dict = {}
        for col in cols: 
            if col == 'ID': 
                temp_dict[col]  = random.randint(1, 10000000)
            elif col == chosen_col: 
                user_feedback = input('Input feedback: ')
                temp_dict[col] = user_feedback
            else: 
                temp_dict[col] = np.nan
        new_df = frag_fb_df.append(temp_dict, ignore_index = True)
        new_df_dict = new_df.to_dict()
        
        ###Then finally save the feedback as the same name json file as before
        with open('fragrance_feedback.json', 'w') as new_fb:
            json.dump(new_df_dict, new_fb)
            
        print('Thank you for your feedback!')
        
        
        #return which_fragrance_num
        return 'We hope you enjoy your recommendations!'
        #return chosen_frag, cols

    
   

In [73]:
give_recommendation(new_dict, df) 

Categories:
1. citrus
2. fruit/vegetable
3. nut
4. flowers
5. white flowers
6. greens/herbs/fougeres
7. spice
8. sweet/gourmand
9. wood/moss
10. resins/balsams
11. musk/amber/animalic
12. beverages
13. natural/synthetic/unusual/popular/weird
Please choose a category from the above (1-13): 1

citrus fragrance notes:
1. tangerine
2. neroli
3. lemon
4. lime
5. bergamot
6. orange
7. grapefruit
8. petitgrain
9. yuzu
10. clementine
11. bitter orange
12. citron
13. lemon verbena
14. bigarade
15. blood orange
16. mandarin orange
17. mandora
18. pomelo
19. lemongrass
20. litsea cubeba
21. lemon balm
22. citruses
23. lemon zest
24. chinotto
25. lemon tree
Please choose a fragrance note from above (1-25): 6
Would you like to add another note? y/n: y
Categories:
1. citrus
2. fruit/vegetable
3. nut
4. flowers
5. white flowers
6. greens/herbs/fougeres
7. spice
8. sweet/gourmand
9. wood/moss
10. resins/balsams
11. musk/amber/animalic
12. beverages
13. natural/synthetic/unusual/popular/weird
Please ch

'We hope you enjoy your recommendations!'

## Future Updates
This notebook is a simple reccommender system, that has the potential to be further worked on to become more complex. For the future, I will accumulate review data that will be used, alongside the cosine similarity, to determine fragrance recommendations that users are more likely to enjoy. 

On a technical level, I will likely convert the function into a class, allowing for recommendations and feedback to be separate, and will also allow for users to come back later and add their feedback after trying one of their recommendations (they will be assigned an ID to input to give feedback, and their previous recommendations will be stored). In addition to this, I plan on adding into a new database of fragrances, so there will be more fragrances that could be recommended. 